# Solar Neighborhood

<img src="https://upload.wikimedia.org/wikipedia/commons/f/fd/Gaia_observes_the_Milky_Way_ESA24305955.jpeg" alt="Drawing" style="width: 400px;"/>

Let's use [Gaia](https://en.wikipedia.org/wiki/Gaia_(spacecraft)) data to look at the properties of stars in the solar neighborhood.

The Gaia Data Release 2 was accompanied by [a paper](https://www.aanda.org/articles/aa/full_html/2018/08/aa32843-18/aa32843-18.html#T2) that makes loads of observational [Hertzsprung-Russell diagrams](https://en.wikipedia.org/wiki/Hertzsprung%E2%80%93Russell_diagram), or "color-magnitude" diagrams.  Gaia has been making [stellar parallax](https://en.wikipedia.org/wiki/Parallax) measurments of stars in the Milky Way

![stellar parallax](https://upload.wikimedia.org/wikipedia/commons/thumb/3/39/14-104-hubble-parallax.png/520px-14-104-hubble-parallax.png)

Measuments of parallax $\varpi$ in arcseconds observed with a 1 AU baseline can be converted to distances in parsecs as simply $d(\mathrm{pc}) = 1/\varpi(\mathrm{arcsec})$ (it's actually what _defines_ the parsec).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Query the Early DR3 Archive

We'll be roughly following the [Observational Hertzsprung-Russell diagram](https://www.aanda.org/articles/aa/full_html/2018/08/aa32843-18/aa32843-18.html#app) paper, including their process for grooming the data to remove astrometric noise, imprecise parallax measurements, etc.  For discussion of the filtering process see [Sec. 2.1](https://www.aanda.org/articles/aa/full_html/2018/08/aa32843-18/aa32843-18.html#S2).

We can do this with a basic query of the [data archive](https://gea.esac.esa.int/archive/), using the GUI to add each condition.  They have also provided the full query in Appendix B, which we'll adopt here with a few changes:

```
SELECT TOP 300000 phot_g_mean_mag+5*log10(parallax)-10 AS mg, bp_rp, parallax FROM gaiaedr3.gaia_source

WHERE parallax_over_error > 10

AND parallax > 10

AND phot_g_mean_flux_over_error>50

AND phot_rp_mean_flux_over_error>20

AND phot_bp_mean_flux_over_error>20

AND phot_bp_rp_excess_factor < 1.3+0.06*power(phot_bp_mean_mag-phot_rp_mean_mag,2)

AND phot_bp_rp_excess_factor > 1.0+0.015*power(phot_bp_mean_mag-phot_rp_mean_mag,2)

AND visibility_periods_used>8

AND astrometric_chi2_al/(astrometric_n_good_obs_al-5)<1.44*greatest(1,exp(-0.4*(phot_g_mean_mag-19.5)))
```

which we can paste directly into an advanced query window (BTW, to get a description of each field see the [Gaia data model](https://gea.esac.esa.int/archive/documentation/GEDR3/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html)).

## HR Diagram

Now let's make our color-magnitude diagram.  For this we need brightness on the y-axis, for which we'll use the total $G$-band magnitude (higher magnitudes are fainter).  On the x-axis we need "color".  We'll be using the diffence in magnitudes when observing with a blue filter ($G_\mathrm{BP}$) and red filter ($G_\mathrm{RP}$).  To give you a sense of what these quantities are, the pass bands for all three filters are shown below.

<img src="https://www.cosmos.esa.int/documents/29201/1645651/GaiaDR2Passbands.png/ba8d3078-0e7a-b316-92d9-cdce748e6484?t=1521129245183" width="500"/>

# Density Plots

It's difficult to see what's going on in the high density regions, so let's work on a density plot.  For this we'll using a Gaussian Kernel Density Estimate (KDE).  Think of this like throughing a little Gaussian distribution down on each data point and adding all of them up to get a smooth estimate of the density of points

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Comparison_of_1D_histogram_and_KDE.png/500px-Comparison_of_1D_histogram_and_KDE.png" width="500"/>